In [1]:
import numpy as np
import pandas as pd

from dotenv import load_dotenv
import os

import unicodedata
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score, f1_score, fbeta_score, roc_curve, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import logging
import json
import re
import string 
import joblib
import warnings
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from mlflow.models.signature import infer_signature

warnings.filterwarnings("ignore")

In [2]:
# Configuración del logging
logging.basicConfig(
    filename="errores_entrenamiento.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


In [3]:
# Load environment variables
try:
    load_dotenv()
    ruta_cst_twcs = os.getenv("customer_support_twitter_twcs")
    logging.info("Environment variables loaded successfully.")
except Exception as e:
    logging.error(f"Error loading environment variables: {e}")
    raise e

In [4]:
# load data
try:
    data_cst_twcs = pd.read_csv(ruta_cst_twcs)
    print(data_cst_twcs.shape)
    logging.info("Data loaded successfully.")
except FileNotFoundError as e:
    logging.error(f"File not found: {ruta_cst_twcs}")
    raise e

(2811774, 7)


In [5]:
data_cst_twcs.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [6]:
# transform the 'inbound' column to int
try:
    data_cst_twcs['inbound'] = data_cst_twcs['inbound'].astype('int')
    logging.info("Data transformed successfully.")
except Exception as e:
    logging.error(f"Error transforming data: {e}")
    raise e

In [7]:
# load stopwords
try:
    nltk.download('punkt')
    #nltk.download('wordnet')
    nltk.download('stopwords')
    english_stopwords = stopwords.words('english')
except Exception as e:
    logging.error(f"Error loading stopwords: {e}")
    raise e

[nltk_data] Downloading package punkt to /home/alejo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/alejo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# split the data into train, validation and test sets
# stratified split to maintain the same proportion of classes in each set
try:
    X_train, X_test, y_train, y_test = train_test_split(data_cst_twcs['text'], data_cst_twcs['inbound'], test_size=0.3, stratify=data_cst_twcs['inbound'], random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train, random_state=42)
    print('X_train: ', X_train.shape)
    print('X_valid: ', X_valid.shape)
    print('X_test: ', X_test.shape)
    print('y_train: ', y_train.shape)
    print('y_valid: ', y_valid.shape)
    print('y_test: ', y_test.shape)
    logging.info("Data split into train, validation and test sets successfully.")
except Exception as e:
    logging.error(f"Error splitting data: {e}")
    raise e

X_train:  (1377768,)
X_valid:  (590473,)
X_test:  (843533,)
y_train:  (1377768,)
y_valid:  (590473,)
y_test:  (843533,)


In [ ]:
# Vectorizar el texto con TF-IDF
#vectorizer = TfidfVectorizer(stop_words=english_stopwords, max_features=100, lowercase=True, token_pattern=r'\b\w+\b')

# Fit the vectorizer on the training data and transform the train, validation and test sets
#X_train = vectorizer.fit_transform(X_train)
#X_valid = vectorizer.transform(X_valid)
#X_test = vectorizer.transform(X_test)

In [9]:
# Set the experiment name
try:
    mlflow.create_experiment("experimento_catboost")
    print("Experimento creado")
    logging.info("Experiment created successfully.")
except:
    mlflow.set_experiment("experimento_catboost")
    print("Experimento ya existe")
    logging.info("Experiment already exists, set to existing experiment.")

Experimento creado


In [ ]:
try:
    print("Iniciando el experimento...")
    with mlflow.start_run():
        # Define y entrena el pipeline
        catboost_params = {
            "iterations": 500,
            "depth": 6,
            "learning_rate": 0.1,
            "verbose": 100
        }

        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words=english_stopwords, max_features=100, lowercase=True, token_pattern=r'\b\w+\b')),
            ('catboost', CatBoostClassifier(**catboost_params))
        ])
        pipeline.fit(X_train, y_train)

        # Inferir signature para input/output
        signature = infer_signature(X_train, y_train)

        # Registra el modelo con un ejemplo de entrada
        input_example = np.array(X_train[:1])  # Toma una muestra como ejemplo de entrada
        mlflow.sklearn.log_model(pipeline, "modelo_catboost", input_example=input_example, signature=signature, registered_model_name="modelo_catboost_prueba")

        # Registra las métricas
        accuracy = pipeline.score(X_test, y_test)
        mlflow.log_metric("accuracy", accuracy)

        # Registra los hiperparámetros del modelo
        mlflow.log_params(catboost_params)

        print(f"Modelo registrado con precisión: {accuracy}")
        logging.info(f"Modelo registrado con precisión: {accuracy}")
except Exception as e:
    logging.error(f"Error during MLflow run: {e}")
    raise e

Iniciando el experimento...
0:	learn: 0.6316539	total: 265ms	remaining: 2m 12s
100:	learn: 0.3638261	total: 16.2s	remaining: 1m 3s
200:	learn: 0.3455131	total: 31s	remaining: 46.1s
300:	learn: 0.3375544	total: 45.4s	remaining: 30s
400:	learn: 0.3330456	total: 1m	remaining: 14.8s
499:	learn: 0.3297846	total: 1m 19s	remaining: 0us


Registered model 'modelo_catboost_prueba' already exists. Creating a new version of this model...
Created version '4' of model 'modelo_catboost_prueba'.


Modelo registrado con precisión: 0.8513466574514571
